In [1]:
import torch
import transformers

In [2]:
import functools
import t5

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [5]:
torch.cuda.is_available()

False

In [4]:
device

device(type='cpu')

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json

In [ ]:
config = transformers.T5Config.from_json_file(json_file="t5-base-config.json")

In [7]:
model = t5.models.HfPyTorchModel("t5-small", "/tmp/hft5/", device)

In [ ]:
# https://github.com/google-research/text-to-text-transfer-transformer/issues/172
model1 = t5.models.HfPyTorchModel(transformers.T5Config(), "/tmp/hft5/", device)

In [ ]:
model2 = t5.models.HfPyTorchModel(config, "/tmp/hft5/", device)

In [ ]:
transformers.T5Config()

In [1]:
config

Available objects for config:
     AliasManager
     DisplayFormatter
     HistoryManager
     IPCompleter
     IPKernelApp
     LoggingMagics
     MagicsManager
     OSMagics
     PrefilterManager
     ScriptMagics
     StoreMagics
     ZMQInteractiveShell


In [8]:
# Evaluate the pre-trained checkpoint, before further fine-tuning
model.eval(
    "glue_cola_v002",
    sequence_length={"inputs": 64, "targets": 4},
    batch_size=128,
)

ValueError: No Task or Mixture found with name: glue_cola_v002

In [ ]:
model.eval.__code__.co_varnames

In [ ]:
model.eval.__defaults__

In [9]:

# Run 1000 steps of fine-tuning
model.train(
    mixture_or_task_name="glue_cola_v002",
    steps=1000,
    save_steps=100,
    sequence_length={"inputs": 64, "targets": 4},
    split="train",
    batch_size=32,
    optimizer=functools.partial(transformers.AdamW, lr=1e-4),
)


ValueError: No Task or Mixture found with name: glue_cola_v002

In [ ]:
# Evaluate after fine-tuning
model.eval(
    "glue_cola_v002",
    checkpoint_steps="all",
    sequence_length={"inputs": 64, "targets": 4},
    batch_size=128,
)

In [ ]:
# Generate some predictions
inputs = [
    "cola sentence: This is a totally valid sentence.",
    "cola sentence: A doggy detail was walking famously.",
]
model.predict(
    inputs,
    sequence_length={"inputs": 32},
    batch_size=2,
    output_file="/tmp/hft5/example_predictions.txt",
)